In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam
import ast
from keras.utils import to_categorical
import joblib

In [ ]:
version = '1'
dataset = pd.read_csv(f'C:/Users/User/Desktop/sum{version}.csv')
dataset = dataset.drop(dataset.columns[0], axis=1)
# Function to convert string coordinates to tuple of floats
def convert_coordinates(coord_string):
    # Remove parentheses and split by comma
    coords = coord_string.strip('()').split(',')
    # Convert each coordinate to float
    return tuple(float(coord) for coord in coords)

# Convert coordinates columns from string to tuple of floats
for col in dataset.columns[:-1]:  # Exclude the last column (labels)
    dataset[col] = dataset[col].apply(convert_coordinates)

# Extract features (coordinates) and labels (classes)
X = dataset.iloc[:, :-1].values  # Features: all columns except the last one
y = dataset.iloc[:, -1].values   # Labels: the last column
# Determine number of classes
num_classes = len(np.unique(y))

# Convert labels to categorical format (required for multi-class classification)
y = to_categorical(y, num_classes=num_classes)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Flatten the coordinate pairs for MLP input (assuming 17 pairs per row)
X_train_flat = np.array([pair for row in X_train for pair in row]).reshape(len(X_train), -1)
X_test_flat = np.array([pair for row in X_test for pair in row]).reshape(len(X_test), -1)
# Standardize the features (optional but recommended for MLPs)
scaler = StandardScaler()
X_train_flat = scaler.fit_transform(X_train_flat)
X_test_flat = scaler.transform(X_test_flat)

# # Define the MLP model using Keras Sequential API
# model = Sequential()
# model.add(Dense(256, input_dim=X_train_flat.shape[1], activation='relu'))
# for _ in range(100):
#     model.add(Dense(256, activation='relu'))
# model.add(Dense(num_classes, activation='softmax'))
model = Sequential([
    Dense(256, input_dim=X_train_flat.shape[1], activation='relu'),  # Input layer with 64 units and ReLU activation
    Dense(256, activation='relu'),                                 # Hidden layer with 64 units and ReLU activation
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(num_classes, activation='softmax')
])
# Compile the model
model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',   # Use categorical crossentropy for multi-class classification
              metrics=['accuracy'])

# Train the model
model.fit(X_train_flat, y_train, epochs=100, batch_size=100, validation_data=(X_test_flat, y_test))

Epoch 1/100


C:\Users\User\anaconda3\envs\DL\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


31/31 [==============================] - 0s 7ms/step - loss: 0.2580 - accuracy: 0.9081 - val_loss: 0.1312 - val_accuracy: 0.9457
Epoch 2/100
31/31 [==============================] - 0s 4ms/step - loss: 0.1008 - accuracy: 0.9644 - val_loss: 0.0720 - val_accuracy: 0.9793
Epoch 3/100
31/31 [==============================] - 0s 4ms/step - loss: 0.0689 - accuracy: 0.9773 - val_loss: 0.0638 - val_accuracy: 0.9780
Epoch 4/100
31/31 [==============================] - 0s 4ms/step - loss: 0.0605 - accuracy: 0.9806 - val_loss: 0.0617 - val_accuracy: 0.9780
Epoch 5/100
31/31 [==============================] - 0s 4ms/step - loss: 0.0507 - accuracy: 0.9815 - val_loss: 0.0762 - val_accuracy: 0.9793
Epoch 6/100
31/31 [==============================] - 0s 4ms/step - loss: 0.0482 - accuracy: 0.9825 - val_loss: 0.0719 - val_accuracy: 0.9780
Epoch 7/100
31/31 [==============================] - 0s 4ms/step - loss: 0.0413 - accuracy: 0.9861 - val_loss: 0.0872 - val_accuracy: 0.9793
Epoch 8/100
31/31 [======

In [3]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Assuming 'model' is your trained MLP model and 'scaler' is your trained scaler
# Evaluate the model
loss, accuracy = model.evaluate(X_test_flat, y_test)
print(f'Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')

# Save the model and scaler
model.save(f'new_mlp_model_{version}.h5')
joblib.dump(scaler, f'new_mlp_scaler_{version}.pkl')

# Predict probabilities for X_test_flat
y_pred_prob = model.predict(X_test_flat)
# Convert probabilities to class predictions
y_pred = np.argmax(y_pred_prob, axis=1)
# Ensure y_test is in the correct format (1D array)
if y_test.ndim > 1:
    y_test = np.argmax(y_test, axis=1)

# Assuming y_test and y_pred are already defined
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Confusion Matrix:")
print(cm)
print("\nAccuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

25/25 [==============================] - 0s 2ms/step - loss: 0.1776 - accuracy: 0.9780
Loss: 0.1776, Accuracy: 0.9780
25/25 [==============================] - 0s 1ms/step
Confusion Matrix:
[[408  10]
 [  7 348]]

Accuracy: 0.9780077619663649
Precision: 0.9720670391061452
Recall: 0.9802816901408451
F1 Score: 0.9761570827489481
